# Model Registry and Deploy
This notebook represents how to deploy a model, already trained in other development platform, with Azure Machine Learning


## Artifactis Gathering
First, you need get all necessary artifacts to deploy your model:
To deploy a model, you need the following:
- **Serialized Model** (pkl, xgb, onnx...)
- **Libraries Environment**: Source code dependencies, which includes the package dependencies required to run the model. (conda.yml or requirements.txt)
- **Entry script** (score.py): This script accepts requests, scores the requests by using the model, and returns the results.


In [2]:
from azureml.core import Workspace
ws = Workspace.from_config()

### Serialized Model
The serialized model we have in this lab is the following file: diabetes_model.pkl

### Library Environments
- Reference: [https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments#create-an-environment](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments#create-an-environment)

In [17]:
from azureml.core.environment import Environment

# From a Conda specification file
myenv = Environment.from_conda_specification(name = "myenv-diabetes",
                                             file_path = "model_artifacts/diabetes_env.yml")

# From a pip requirements file
#myenv = Environment.from_pip_requirements(name = "myenv", file_path = "path-to-pip-requirements-file")      

#Register my env in my workspace
myenv.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220314.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv-diabetes",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"

### Entry Script
- Example: [https://github.com/Azure/azureml-examples/blob/main/cli/endpoints/online/model-1/onlinescoring/score.py](https://github.com/Azure/azureml-examples/blob/main/cli/endpoints/online/model-1/onlinescoring/score.py)
We already have our score.py (score_diabetes.py), the following is just the test of this code locally.

In [11]:
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    #model_path = os.path.join(Model.get_model_path('diabetes_model'), 'diabetes_model.pkl')
    model_path = "model_artifacts/diabetes_model.pkl"
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)


In [12]:
init()

In [13]:
input_data = '{"data": [[2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22], [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45]]}'
run(input_data)

'["diabetic", "not-diabetic"]'

## Deploy the Model
To deploy the model we need first register and then deploy

### Register a model
Now let's register the model to then deploy it.

In [15]:
from azureml.core import Model
import sklearn

# Register the model
model_path = "model_artifacts"
model_azure = Model.register(model_path=model_path, 
               workspace = ws,
               model_name='diabetes_model_testeltm',
               tags={'Training context':'Inline Training', 'area': 'diabetes', 'type': 'classification', 'algorithm' : 'DecisionTreeClassifier'},
               properties={'AUC': '0.87', 
                   'Accuracy': '0.91'},
                model_framework = Model.Framework.SCIKITLEARN,
                model_framework_version = sklearn.__version__,
                description='Decision tree model to predict diabetes.')

Registering model diabetes_model_testeltm


In [16]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

diabetes_model version 34


### Deployment configuration

Now you're ready to deploy. We'll deploy the container a service named **diabetes-service**. The deployment process includes the following steps:

1. Define an inference configuration, which includes the scoring and environment files required to load and use the model.
2. Define a deployment configuration that defines the execution environment in which the service will be hosted. In this case, an Azure Container Instance.
3. Deploy the model as a web service.
4. Verify the status of the deployed service.

> **More Information**: For more details about model deployment, and options for target execution environments, see the [documentation](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where).

Deployment will take some time as it first runs a process to create a container image, and then runs a process to create a web service based on the image. When deployment has completed successfully, you'll see a status of **Healthy**.

At this moment we will define the infrastructure we will use do deploy our model.
- ACI (Azure Container Instance): Recommended in QA environments
- AKS (Azure Container Registry): Recommended in Prod environments



#### Deploy on ACI

In [21]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(entry_script='model_artifacts/score_diabetes.py',
                                   environment=myenv.get(ws,'myenv-diabetes'))

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 2, auth_enabled=True)

service_name = "diabetes-qa-endpoint"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-07-05 20:46:54+00:00 Creating Container Registry if not exists.
2022-07-05 20:46:55+00:00 Registering the environment.
2022-07-05 20:46:56+00:00 Use the existing image.
2022-07-05 20:46:56+00:00 Generating deployment configuration.
2022-07-05 20:46:57+00:00 Submitting deployment to compute.
2022-07-05 20:47:01+00:00 Checking the status of deployment diabetes-qa-endpoint..
2022-07-05 20:49:37+00:00 Checking the status of inference endpoint diabetes-qa-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


Hopefully, the deployment has been successful and you can see a status of **Healthy**. If not, you can use the following code to get the service logs to help you troubleshoot.

In [22]:
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

2022-07-05T20:49:31,063491700+00:00 - iot-server/run 
2022-07-05T20:49:31,072232700+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-07-05T20:49:31,072485900+00:00 - rsyslog/run 
2022-07-05T20:49:31,083858600+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-07-05T20:49:31,270525800+00:00 - iot-server/finish 1 0
2022-07-05T20:49:31,293546000+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (74)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 102
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2022-07-05 20:49:34,783 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2022-07-05 20:49:34,783 | root | INFO | Starting up request id generator
2022-07-05 20:49:34,784 | root | INFO | Sta

## Use the web service

With the service deployed, now you can consume it from a HTTP request, sending the patient data in JSON format, and receive back the predicted class(es).

In [24]:
import urllib.request
import json
import os
import ssl


# Request data goes here
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]
print ('Patients: {}'.format(x_new))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

body = str.encode(input_json)

url = 'http://12feb218-28cb-4a18-b03b-ddc1d71eac8c.eastus.azurecontainer.io/score'
#api_key = '' # Replace this with the API key for the web service , 'Authorization':('Bearer '+ api_key)
headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read())
    

Patients: [[2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22], [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45]]
b'"[\\"diabetic\\", \\"not-diabetic\\"]"'


#### Deploy on AKS